# LAB 3: How to setup a project from Scratch

In [1]:
!rm -rf speech-emotion-recognition-25

# Step 1: Clone your project from Github

In [2]:
#main
#!git clone https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

#             nome branch

!git clone -b DatasetSetup https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

Cloning into 'speech-emotion-recognition-25'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 103 (delta 46), reused 62 (delta 18), pack-reused 11 (from 2)
Receiving objects: 100% (103/103), 32.05 KiB | 10.68 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [3]:
!ls

sample_data  speech-emotion-recognition-25


In [4]:
# %cd mldl_project_skeleton
%cd speech-emotion-recognition-25

/content/speech-emotion-recognition-25


In [5]:
!ls

checkpoints  dataset  models	 requirements.txt		  train.py
data	     eval.py  README.md  SpeechEmotionRecnognition.ipynb  utils


# Step 2: Packages Installation


In [6]:
%pip install -r requirements.txt

# Step 3: Dataset Setup
## Different options
- First one is downloading using a script that places the data in the download folder (usually recommended)
- Second one is uploading the dataset to your personal/institutional Google Drive and load it from there ([Read More](https://saturncloud.io/blog/google-colab-how-to-read-data-from-my-google-drive/))
- Place the download script directly here on colab

You are free to do as you please in this phase.


In [7]:
!python utils/download_dataset.py

--- 1. Configurazione Kaggle ---
Kaggle configurato con successo.

--- Download RAVDESS ---
Contatto KaggleHub per scaricare: uwrfkaggler/ravdess-emotional-speech-audio...
Using Colab cache for faster access to the 'ravdess-emotional-speech-audio' dataset.
✓ Dataset scaricato nella cache di sistema: /kaggle/input/ravdess-emotional-speech-audio
Copia dei file nella cartella di lavoro: ./ravdess...
RAVDESS pronto in: ./ravdess
Numero totale di file copiati: 2880

--- Download IEMOCAP ---
Contatto KaggleHub per scaricare: dejolilandry/iemocapfullrelease...
100% 11.5G/11.5G [02:09<00:00, 95.9MB/s]
Extracting files...
✓ Dataset scaricato nella cache di sistema: /root/.cache/kagglehub/datasets/dejolilandry/iemocapfullrelease/versions/1
Inizio la copia selettiva (cercando: 'Impro')...
IEMOCAP pronto in: ./iemocap
Numero totale di file copiati: 37247

RIEPILOGO DOWNLOAD
RAVDESS: ✅ Successo
IEMOCAP: ✅ Successo

🎉 Tutti i dataset sono stati scaricati con successo!


In [ ]:
# ============================================================================
# ESPLORAZIONE STRUTTURA DATASET (VERSIONE COMPLETA)
# ============================================================================

from pathlib import Path
from collections import defaultdict

def analyze_dataset_structure_detailed(dataset_path, dataset_name):
    """Analizza e mostra la struttura COMPLETA di un dataset"""
    print("=" * 100)
    print(f"📁 STRUTTURA DATASET COMPLETA: {dataset_name.upper()}")
    print("=" * 100)
    
    path = Path(dataset_path)
    
    if not path.exists():
        print(f"❌ Directory non trovata: {dataset_path}")
        return
    
    # 1. STATISTICHE GENERALI
    print("\n📊 STATISTICHE GENERALI:")
    print("-" * 100)
    
    total_files = 0
    total_size = 0
    file_extensions = defaultdict(int)
    dir_count = 0
    
    for item in path.rglob('*'):
        if item.is_file():
            total_files += 1
            total_size += item.stat().st_size
            ext = item.suffix.lower() or '[no extension]'
            file_extensions[ext] += 1
        elif item.is_dir():
            dir_count += 1
    
    print(f"  • Totale cartelle: {dir_count}")
    print(f"  • Totale file: {total_files}")
    print(f"  • Dimensione totale: {total_size / (1024**3):.2f} GB")
    print(f"  • Tipi di file:")
    for ext, count in sorted(file_extensions.items(), key=lambda x: x[1], reverse=True):
        print(f"      {ext}: {count} file(s)")
    
    # 2. ALBERO COMPLETO DELLE DIRECTORY
    print("\n📂 ALBERO COMPLETO DELLE CARTELLE:")
    print("-" * 100)
    
    def print_tree_complete(directory, prefix="", max_depth=10, current_depth=0):
        """Stampa l'intero albero senza limitazioni"""
        if current_depth >= max_depth:
            return
        
        try:
            items = sorted(directory.iterdir(), key=lambda x: (not x.is_dir(), x.name))
            dirs = [item for item in items if item.is_dir()]
            files = [item for item in items if item.is_file()]
            
            # Mostra TUTTE le directory
            for i, item in enumerate(dirs):
                is_last_dir = (i == len(dirs) - 1) and len(files) == 0
                print(f"{prefix}{'└── ' if is_last_dir else '├── '}📁 {item.name}/")
                
                extension = "    " if is_last_dir else "│   "
                print_tree_complete(item, prefix + extension, max_depth, current_depth + 1)
            
            
        except PermissionError:
            print(f"{prefix}❌ Accesso negato")
    
    print_tree_complete(path)
    
    # 3. ELENCO DIRECTORY PER LIVELLO
    print("\n\n📊 GERARCHIA DIRECTORY PER LIVELLO:")
    print("-" * 100)
    
    level_dirs = defaultdict(list)
    for item in path.rglob('*'):
        if item.is_dir():
            depth = len(item.relative_to(path).parts)
            relative_path = item.relative_to(path)
            level_dirs[depth].append(str(relative_path))
    
    for level in sorted(level_dirs.keys()):
        print(f"\n🔹 Livello {level} ({len(level_dirs[level])} cartelle):")
        for dir_path in sorted(level_dirs[level])[:20]:  # Mostra prime 20
            print(f"     • {dir_path}")
        if len(level_dirs[level]) > 20:
            print(f"     ... e altri {len(level_dirs[level]) - 20} directory")
    
    # 4. CONTENUTO DETTAGLIATO PER SESSION (SE IEMOCAP)
    if "iemocap" in dataset_name.lower():
        print("\n\n🎙️ DETTAGLI IEMOCAP - CONTENUTO PER SESSION:")
        print("-" * 100)
        
        sessions = sorted([d for d in path.iterdir() if d.is_dir() and d.name.startswith("Session")])
        for session in sessions:
            print(f"\n  {session.name}:")
            subdirs = sorted([d for d in session.iterdir() if d.is_dir()])
            for subdir in subdirs:
                file_count = len(list(subdir.glob('*')))
                print(f"    ├── {subdir.name}/ ({file_count} items)")
                # Mostra alcuni file esempi
                sample_files = list(subdir.glob('*'))[:5]
                for f in sample_files:
                    if f.is_file():
                        size = f.stat().st_size / (1024**2) if f.stat().st_size > 1024**2 else f.stat().st_size / 1024
                        size_unit = "MB" if f.stat().st_size > 1024**2 else "KB"
                        print(f"    │   ├── {f.name} ({size:.2f} {size_unit})")
                    else:
                        print(f"    │   ├── {f.name}/")
                if len(sample_files) < file_count:
                    print(f"    │   └── ... altri {file_count - len(sample_files)} items")
    
    print("\n" + "=" * 100 + "\n")


# ============================================================================
# ANALISI RAVDESS
# ============================================================================

print("\n" + "🎵" * 40)
print("ANALISI DATASET RAVDESS")
print("🎵" * 40 + "\n")

analyze_dataset_structure_detailed("./ravdess", "RAVDESS")


# ============================================================================
# ANALISI IEMOCAP
# ============================================================================

print("\n" + "🎙️" * 40)
print("ANALISI DATASET IEMOCAP")
print("🎙️" * 40 + "\n")

analyze_dataset_structure_detailed("./iemocap", "IEMOCAP")


NameError: name 'dirs' is not defined

# Step 3.5: Improvised Vocal Recording

Model IEMOCAP dataset to extract only improvised audio. 

In [9]:
## insert code here 

# Step 4: Train your model and visualize training

In [10]:
#%env WANDB_API_KEY="7ade30086de7899bed412e3eb5c2da065c146f90"

In [11]:
#!python train.py

# Step 5: Evaluate your model

1.   List item
2.   List item



In [12]:
#!python eval.py